<a href="https://colab.research.google.com/github/filiperobotic/cursoDL/blob/master/codes/aula8/Pokemons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/filiperobotic/cursoDL.git

Cloning into 'cursoDL'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25015 (delta 8), reused 0 (delta 0), pack-reused 24995
Receiving objects: 100% (25015/25015), 945.73 MiB | 36.42 MiB/s, done.
Resolving deltas: 100% (1039/1039), done.
Checking out files: 100% (31176/31176), done.


In [0]:

from keras.models import Sequential, Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.layers.convolutional import Conv2D

from sklearn.preprocessing import LabelBinarizer
from keras import applications
from keras.layers import Input
from keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from cursoDL.codes.aula4.preprocessing import ImageToArrayPreprocessor
from cursoDL.codes.aula4.preprocessing import SimplePreprocessor
from cursoDL.codes.aula4.datasets import SimpleDatasetLoader
#from pyimagesearch.nn.conv import ShallowNet
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np


In [0]:
def build(width, height, depth, classes):
  
  model = Sequential()
  inputShape = (height, width, depth)
  
  
  model.add(Conv2D(128, (5, 5), input_shape=inputShape, padding="same"))
  model.add(Activation("relu"))
  model.add(Conv2D(64, (3, 3), padding="same"))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(32, (5, 5), padding="same"))
  model.add(Activation("relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  model.add(Dropout(0.25))
            
  model.add(Flatten())
  model.add(Dense(500))
  model.add(Activation("relu"))
  
  #model.add(Dense(250))
  #model.add(Activation("relu"))

  # softmax classifier
  model.add(Dense(classes))
  model.add(Activation("softmax"))

  # return the constructed network architecture
  return model

In [63]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
#from pyimagesearch.simplenet import SimpleNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from imutils import build_montages
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os


arg_dataset = "cursoDL/imagens/datasets/pokemons"
arg_epochs = 15


print("[INFO] loading images...")
imagePaths = list(paths.list_images(arg_dataset))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# load the image, convert it to grayscale, and resize it to be a
	# fixed 100X75 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
	#image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (100, 75))
	data.append(image)
	labels.append(label)
  
  
  # convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

# reshape the data matrix so that it explicity includes a channel
# dimension
data = data.reshape((data.shape[0], data.shape[1], data.shape[2], 3))
print (data.shape)
# encode the labels (which are currently strings) as integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# transform the labels into vectors in the range [0, classes],
# generating a vector for each label, where the index of the label
# is set to '1' and all other entries are set to '0' -- this process
# is called "one-hot encoding"
labels = np_utils.to_categorical(labels, 5)

# partition the data into training and testing splits using 60% of
# the data for training and the remaining 40% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.40, stratify=labels, random_state=42)

print("[INFO] compiling model...")
opt = Adam(lr=1e-4, decay=1e-4 / arg_epochs)
model = build(width=100, height=75, depth=3, classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
print("[INFO] training network for {} epochs...".format(
	arg_epochs))
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	batch_size=32, epochs=arg_epochs, verbose=1)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=le.classes_))

[INFO] loading images...
(1167, 75, 100, 3)
[INFO] compiling model...
[INFO] training network for 15 epochs...
Train on 700 samples, validate on 467 samples
Epoch 1/15
700/700 [==============================] - 6s 9ms/step - loss: 0.4783 - acc: 0.8000 - val_loss: 0.4208 - val_acc: 0.8000
Epoch 2/15
700/700 [==============================] - 3s 4ms/step - loss: 0.3285 - acc: 0.8523 - val_loss: 0.2743 - val_acc: 0.8848
Epoch 3/15
700/700 [==============================] - 3s 4ms/step - loss: 0.2245 - acc: 0.9160 - val_loss: 0.2343 - val_acc: 0.9092
Epoch 4/15
700/700 [==============================] - 3s 4ms/step - loss: 0.1715 - acc: 0.9397 - val_loss: 0.2022 - val_acc: 0.9242
Epoch 5/15
700/700 [==============================] - 3s 4ms/step - loss: 0.1434 - acc: 0.9523 - val_loss: 0.1559 - val_acc: 0.9503
Epoch 6/15
700/700 [==============================] - 3s 4ms/step - loss: 0.1159 - acc: 0.9637 - val_loss: 0.1504 - val_acc: 0.9465
Epoch 7/15
700/700 [==============================]

In [47]:
#TRANSFER LEARNING

img_width, img_height = 48, 48

model = applications.VGG16(weights = "imagenet", include_top=False, input_shape=(img_width, img_height, 3))


for layer in model.layers:
  layer.trainable = False
  
#Adding custom layers
x = model.output
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

# creating the final model
model_final = Model(input = model.input, output = predictions)
model = model_final

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [48]:
args_dataset="cursoDL/imagens/datasets/pokemons"

arg_epochs = 15


print("[INFO] loading images...")
imagePaths = list(paths.list_images(arg_dataset))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	# load the image, convert it to grayscale, and resize it to be a
	# fixed 100X75 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
	#image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (48, 48))
	data.append(image)
	labels.append(label)
  
  
  # convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

# reshape the data matrix so that it explicity includes a channel
# dimension
data = data.reshape((data.shape[0], data.shape[1], data.shape[2], 3))

# encode the labels (which are currently strings) as integers
le = LabelEncoder()
labels = le.fit_transform(labels)

# transform the labels into vectors in the range [0, classes],
# generating a vector for each label, where the index of the label
# is set to '1' and all other entries are set to '0' -- this process
# is called "one-hot encoding"
labels = np_utils.to_categorical(labels, 5)

# partition the data into training and testing splits using 60% of
# the data for training and the remaining 40% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.40, stratify=labels, random_state=42)

# initialize the optimizer and model
print("[INFO] compiling model...")
#opt = SGD(lr=0.001)
opt = Adam(lr=1e-4, decay=1e-4 / arg_epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
num_epocas = 100
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=32, epochs=num_epocas, verbose=1)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1),
	target_names=["bulbasaur", "charmander", "mewtwo", "pikachu","squirtle"]))

[INFO] loading images...
[INFO] compiling model...
[INFO] training network...
Train on 700 samples, validate on 467 samples
Epoch 1/100
700/700 [==============================] - 3s 4ms/step - loss: 1.6314 - acc: 0.2529 - val_loss: 1.5554 - val_acc: 0.3041
Epoch 2/100
700/700 [==============================] - 1s 2ms/step - loss: 1.5208 - acc: 0.3229 - val_loss: 1.4767 - val_acc: 0.4218
Epoch 3/100
700/700 [==============================] - 1s 1ms/step - loss: 1.4583 - acc: 0.4000 - val_loss: 1.4090 - val_acc: 0.4882
Epoch 4/100
700/700 [==============================] - 1s 1ms/step - loss: 1.3718 - acc: 0.4829 - val_loss: 1.3462 - val_acc: 0.5589
Epoch 5/100
700/700 [==============================] - 1s 1ms/step - loss: 1.2927 - acc: 0.5700 - val_loss: 1.2901 - val_acc: 0.5675
Epoch 6/100
700/700 [==============================] - 1s 1ms/step - loss: 1.2485 - acc: 0.5843 - val_loss: 1.2401 - val_acc: 0.5931
Epoch 7/100
700/700 [==============================] - 1s 2ms/step - loss: 1.1